In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
# import plotly.express as px
from dash.dependencies import Input, Output
import dash_table as dt

data_path = 'data/inventory.csv'
df = pd.read_csv(data_path)
df.head()

,product_type,etat,cost_price,country,tenant_id,stock_type,quantite
0,WowSolar Controller,Neuf,NaN,Senegal,1,personal,3
1,Hairclipper component,Neuf,9830.00000,Senegal,1,personal,44
2,WowSolar Controller,Endommagé,155305.69540,Senegal,1,multi_manager,54
3,SKH Controller,Endommagé,37707.47966,Senegal,1,personal,36
4,WowSolar Controller,Endommagé,30358.00000,Senegal,1,personal,11


In [42]:
df.isnull().sum()

stock_id          0
created_at        0
serial            0
etat              0
cost_price       29
stock_manager     0
tenant_id         0
stock_type        0
locality          0
product_type      0
stock_name        0
country           0
dtype: int64

In [52]:
df.fillna(0,inplace=True)

In [53]:
df.isnull().sum()

stock_id         0
created_at       0
serial           0
etat             0
cost_price       0
stock_manager    0
tenant_id        0
stock_type       0
locality         0
product_type     0
stock_name       0
country          0
dtype: int64

In [54]:
df['product_type'].unique()

array(['SKH Controller', 'WowSolar Controller', 'PG TEST',
       'Hairclipper component', 'Radio component', 'D100 Controller',
       'Oolu TV Screen 24', 'WowSolar TV Screen 22'], dtype=object)

In [48]:
df.dtypes

stock_id           int64
created_at        object
serial            object
etat              object
cost_price       float64
stock_manager     object
tenant_id          int64
stock_type        object
locality          object
product_type      object
stock_name        object
country           object
dtype: object

In [55]:
import  psycopg2
from psycopg2 import Error
from sqlalchemy import text


In [56]:
sql_file = open('data/inventory.sql')
sql_text = sql_file.read()
# Connection to oolusolar database
try:
    connection = psycopg2.connect(user = 'chartio_read_only_user',
                                  password = '2ZVF01USUWTKV3K9JJFY',
                                  host = 'oolu-main-postgresql.cfa4plgxjs0u.eu-central-1.rds.amazonaws.com',
                                  port ='5432',
                                  database = 'oolusolar_analytics')
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print('You are Successfully connected to - ',record, '\n')
except (Exception, Error) as error:
    print(" Connection failed, try again", error)
    cursor.close()

data  = pd.read_sql_query(sql_text,connection)
cursor.close()
connection.close()

You are Successfully connected to -  ('PostgreSQL 9.5.24 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',) 



In [57]:
data.head()

,stock_id,created_at,serial,etat,cost_price,stock_manager,tenant_id,stock_type,locality,product_type,stock_name,country
0,90,2018-06-26 08:28:09,5602453,Endommagé,37707.47966,Thies,1,multi_manager,THIES,SKH Controller,Thies,Senegal
1,90,2017-07-31 15:09:16,8039646,Endommagé,37707.47966,Thies,1,multi_manager,THIES,SKH Controller,Thies,Senegal
2,89,2017-02-26 17:56:19,6975924,Endommagé,37707.47966,Fatick,1,multi_manager,FATICK,SKH Controller,Fatick,Senegal
3,89,2017-03-27 18:38:56,7393648,Endommagé,37707.47966,Fatick,1,multi_manager,FATICK,SKH Controller,Fatick,Senegal
4,89,2017-05-07 14:17:04,9069840,Endommagé,37707.47966,Fatick,1,multi_manager,FATICK,SKH Controller,Fatick,Senegal


In [58]:
max(data.created_at)

Timestamp('2021-02-14 19:49:51.809653')

In [59]:
import calendar
a = calendar.monthcalendar(2021,3)
a

[[1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19, 20, 21],
 [22, 23, 24, 25, 26, 27, 28],
 [29, 30, 31, 0, 0, 0, 0]]

In [62]:
df_1 = data.groupby(['product_type','etat','cost_price', 'stock_type'],as_index=False).agg({'serial':'count'})

In [63]:
df_1

,product_type,etat,cost_price,stock_type,serial
0,Hairclipper component,Neuf,9830.00000,multi_manager,3
1,Hairclipper component,Neuf,9830.00000,personal,44
2,Radio component,Neuf,7509.00000,multi_manager,3
3,Radio component,Neuf,7509.00000,personal,12
4,SKH Controller,Endommagé,33435.00000,multi_manager,394
5,SKH Controller,Endommagé,33435.00000,personal,13
6,SKH Controller,Endommagé,37707.00000,multi_manager,416
7,SKH Controller,Endommagé,37707.47966,multi_manager,3775
8,SKH Controller,Endommagé,37707.47966,personal,36
9,WowSolar Controller,Endommagé,30358.00000,multi_manager,26
